In [1]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression, Ridge, LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostRegressor,  BaggingRegressor, ExtraTreesRegressor, GradientBoostingRegressor, RandomForestRegressor
import xgboost 

from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
import altair as alt
import datetime as dt

In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf  # for data
from pandas_datareader import data as pdr
yf.pdr_override()

In [35]:
def read_data(stock='TSLA', period = "60d", interval = "60m"):
    '''
    reading the data
    '''


    df = pdr.get_data_yahoo(stock,period = period,

            # fetch data by interval (including intraday if period < 60 days)
            # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
            # (optional, default is '1d')
            interval = interval,)


    return df

In [36]:
def read_data_enhanced(stock='TSLA', period = "60d", interval = "2m"):
    '''
    reading the data
    '''


    dfx = pdr.get_data_yahoo(stock,period = period,

            # fetch data by interval (including intraday if period < 60 days)
            # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
            # (optional, default is '1d')
            interval = interval,).reset_index()



    df_nasaq = read_data(stock='NDAQ', period = period, interval = interval).reset_index()[['Datetime','Open','Adj Close']]
        
    df_nasaq['fn'] = ((df_nasaq['Open']/df_nasaq['Adj Close'].shift(1))-1)*100
    df_nasaq= df_nasaq[['Datetime','fn']]
    
    
    df_sp = read_data(stock='^GSPC', period = period, interval = interval).reset_index()[['Datetime','Open','Adj Close']]
    df_sp['fs'] = ((df_sp['Open']/df_sp['Adj Close'].shift(1))-1)*100
    df_sp= df_sp[['Datetime','fs']]
    

    dfx = pd.merge(dfx,df_nasaq,on = ['Datetime'], how = 'left')
    
    dfx = pd.merge(dfx,df_sp,on = ['Datetime'], how = 'left')
    
    
        
    return dfx

In [55]:
df1 = read_data(stock='TSLA').reset_index()

[*********************100%***********************]  1 of 1 completed


In [56]:
df2 = df1.copy()

In [57]:
# here is the tail of the df which has the most recent data
df1.head()

,Datetime,Open,High,Low,Close,Adj Close,Volume
0,2020-08-24 09:30:00-04:00,423.800110,425.799988,385.503998,397.815979,397.815979,8308551
1,2020-08-24 10:30:00-04:00,397.658386,404.662018,393.000000,398.000000,398.000000,3129966
2,2020-08-24 11:30:00-04:00,398.289001,412.497986,397.600006,408.158020,408.158020,2697214
3,2020-08-24 12:30:00-04:00,408.157013,409.724030,403.408020,405.200012,405.200012,1547673
4,2020-08-24 13:30:00-04:00,405.098389,405.667511,400.127991,401.886017,401.886017,1547734


In [58]:
df2 = read_data_enhanced(stock='TSLA')
df2.head()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Datetime,Open,High,Low,Close,Adj Close,Volume,fn,fs
0,2020-08-31 09:30:00-04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-10-05 09:30:00-04:00,423.350006,425.500000,421.220001,424.690002,424.690002,2672089.0,NaN,NaN
2,2020-10-05 09:32:00-04:00,424.829987,425.950012,423.290009,424.383911,424.383911,656005.0,0.369535,0.006516
3,2020-10-05 09:34:00-04:00,424.464996,425.589996,423.399994,423.890015,423.890015,535530.0,-0.145962,-0.001770
4,2020-10-05 09:36:00-04:00,423.917786,427.570007,423.769989,427.375000,427.375000,738799.0,-0.003877,-0.003549


### Models

In [41]:
# We have the follwoing model already established

In [42]:
def test_stock(dfx,intra =True):
    """
    predict the price for the closing price for the bext inerval. 
    
    inputs:
    ------
    - stock: string, symobol for the stick, eg. 'TSLA'
    - intra : Boolean, if True then the period and interval should be 1m, 5m, etc, if False it should be 1d, 7d, etc
    - period : total period
    - interval : interval for the data
    
    """
    df = dfx.copy()
    #df['day_of_the_week'] = df['Datetime'].dt.dayofweek
    #df['day_of_month'] = df['Datetime'].apply(lambda x:int(str(x)[8:10]))


    
    #print(len(df))
    print(df.tail(1)['Datetime'])
    old = df.tail(1).loc[:,'Open'].to_numpy()[0]
    
    if intra ==True:
            df['Datetime'] = pd.to_datetime(df['Datetime'])
            df['Datetime']=df['Datetime'].map(dt.datetime.toordinal)
            df['SMA_1'] = df['Adj Close'].rolling(window = 26).mean()
            df['SMA_2'] = df['Adj Close'].rolling(window = 50).mean()
            df['status_sma_lower'] = np.where(df['Adj Close'] < df['SMA_1'],'below_movavg','normal')
            df['status_sma_upper'] = np.where(df['Adj Close'] > df['SMA_2'],'above_movavg','normal')
    
    elif intra ==False:
            df['Date'] = pd.to_datetime(df['Date'])
            df['Date']=df['Date'].map(dt.datetime.toordinal)
#             df['SMA_1'] = df['Adj Close'].rolling(window = 50).mean()
#             df['SMA_2'] = df['Adj Close'].rolling(window = 100).mean()
#             df['status_sma_lower'] = np.where(df['Adj Close'] < df['SMA_1'],'below_movavg','normal')

    

     
    df['High'] = df['High'].shift(1)
    df['Low'] = df['Low'].shift(1)
    df['Volume'] = df['Volume'].shift(1)
    
    df['Middle Band'] =df['Adj Close'].rolling(window=26).mean()
    df['Upper Band'] = df['Middle Band'] + 1.96*df['Adj Close'].rolling(window=20).std()
    df['Lower Band'] = df['Middle Band'] - 1.96*df['Adj Close'].rolling(window=20).std()
    df['status_lower'] = np.where(df['Adj Close'] < df['Lower Band'],'below_ballinger','normal')
    df['status_upper'] = np.where(df['Adj Close'] > df['Upper Band'],'above_ballinger','normal')
    #df['f01'] = df['Close']/df['Open']-1


    df = df.dropna()
    df = df.drop(columns = ['Close'])
    y = df['Adj Close']
    X = df.drop(columns =['Adj Close','Volume' ,'Low','High'])
    X_train, X_test, y_train, y_test = train_test_split(X,y,shuffle= False)
        # first transformer for the numeric features
    numeric_features = list(set(list(X_train.columns)) - set(['status_lower','status_upper','status_sma_lower','status_sma_upper']))
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler()),
       ])
    # now a taransformer for the categorical features
    categorical_features = ['status_lower','status_upper']
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))])
    # creating a preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)])
    
    ridge_model = Ridge()
    # include the preprocessor and the model in one pipeline.
    # Now we have a full prediction pipeline.
    reg_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                          ('Regressor', ridge_model)])

    # finally we will pass the pipe line to gridsearchcv to find the optimum paramters for the model
    param_grid = {
        'Regressor__alpha':[0.1,0.25,0.4],
    }



    search = GridSearchCV(reg_pipeline,param_grid,cv = 5)

    # fitting the model
    search.fit(X_train, y_train)

    # printing the first parameter
    #rint(search.best_params_)
    print("model score: %.3f" % search.score(X_test, y_test))
    X_train = X[:-1]
    y_train = y[:-1]
    X_test = X[-1:]
    y_test = y[-1:]
    search.fit(X_train,y_train)
    y_pred = search.predict(X_train)
    print('error',np.sqrt(mean_squared_error(y_train,y_pred)))
    y_test = search.predict(X_test)[0]
    diff = y_test - old
   
    print('old price = ',old)
    print('predicted = ',y_test)
    print('difference = ', diff)
    



In [43]:
# now let's see the power of feature engineering:
#we will add the daily return as a feature for this model

In [44]:
def test_stock_w_feature(dfx,intra =True):
    """
    predict the price for the closing price for the bext inerval. 
    
    inputs:
    ------
    - stock: string, symobol for the stick, eg. 'TSLA'
    - intra : Boolean, if True then the period and interval should be 1m, 5m, etc, if False it should be 1d, 7d, etc
    - period : total period
    - interval : interval for the data
    
    """
    #print(len(df))


    df = dfx.copy()
    
    # for now the following do not add any predictiv power
    #df['day_of_the_week'] = df['Datetime'].dt.dayofweek
    #df['day_of_month'] = df['Datetime'].apply(lambda x:int(str(x)[8:10]))
    

        


    
    df['f01'] = ((df['Open']/df['Close'].shift(1))-1)*100
    old = df.tail(1).loc[:,'Open'].to_numpy()[0]
    
    if intra ==True:
            df['Datetime'] = pd.to_datetime(df['Datetime'])
            df['Datetime']=df['Datetime'].map(dt.datetime.toordinal)
            df['SMA_1'] = df['Adj Close'].rolling(window = 26).mean()
            df['SMA_2'] = df['Adj Close'].rolling(window = 50).mean()
            df['status_sma_lower'] = np.where(df['Adj Close'] < df['SMA_1'],'below_movavg','normal')
            df['status_sma_upper'] = np.where(df['Adj Close'] > df['SMA_2'],'above_movavg','normal')
    
    elif intra ==False:
            df['Date'] = pd.to_datetime(df['Date'])
            df['Date']=df['Date'].map(dt.datetime.toordinal)
#             df['SMA_1'] = df['Adj Close'].rolling(window = 50).mean()
#             df['SMA_2'] = df['Adj Close'].rolling(window = 100).mean()
#             df['status_sma_lower'] = np.where(df['Adj Close'] < df['SMA_1'],'below_movavg','normal')
#             df['status_sma_upper'] = np.where(df['Adj Close'] > df['SMA_2'],'above_movavg','normal')

     
    df['High'] = df['High'].shift(1)
    df['Low'] = df['Low'].shift(1)
    df['Volume'] = df['Volume'].shift(1)
    df['Middle Band'] =df['Adj Close'].rolling(window=26).mean()
    df['Upper Band'] = df['Middle Band'] + 1.96*df['Adj Close'].rolling(window=20).std()
    df['Lower Band'] = df['Middle Band'] - 1.96*df['Adj Close'].rolling(window=20).std()
    df['status_lower'] = np.where(df['Adj Close'] < df['Lower Band'],'below_ballinger','normal')
    df['status_upper'] = np.where(df['Adj Close'] > df['Upper Band'],'above_ballinger','normal')
    #df['f01'] = df['Close']/df['Open']-1


    df = df.dropna()
    df = df.drop(columns = ['Close'])
    y = df['Adj Close']
    X = df.drop(columns =['Adj Close','Volume' ,'Low','High'])
    X_train, X_test, y_train, y_test = train_test_split(X,y,shuffle= False)
        # first transformer for the numeric features
    numeric_features = list(set(list(X_train.columns)) - set(['status_lower','status_upper','status_sma_lower','status_sma_upper']))
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler()),
       ])
    # now a taransformer for the categorical features
    categorical_features = ['status_lower','status_upper']
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))])
    # creating a preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)])
    
    ridge_model = Ridge()
    # include the preprocessor and the model in one pipeline.
    # Now we have a full prediction pipeline.
    reg_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                          ('Regressor', ridge_model)])

    # finally we will pass the pipe line to gridsearchcv to find the optimum paramters for the model
    param_grid = {
        'Regressor__alpha':[0.1,0.25,0.4],
    }



    search = GridSearchCV(reg_pipeline,param_grid,cv = 5)

    # fitting the model
    search.fit(X_train, y_train)

    # printing the first parameter
    #rint(search.best_params_)
    print("model score: %.3f" % search.score(X_test, y_test))
    X_train = X[:-1]
    y_train = y[:-1]
    X_test = X[-1:]
    y_test = y[-1:]
    search.fit(X_train,y_train)
    y_pred = search.predict(X_train)
    print('error',np.sqrt(mean_squared_error(y_train,y_pred)))
    y_test = search.predict(X_test)[0]
    diff = y_test - old
   
    print('old price = ',old)
    print('predicted = ',y_test)
    print('difference = ', diff)
    




### Performance Comparision

In [45]:
# let's predict he clost price for TSLA after 5 minutes
test_stock(df1, intra = True)

417   2020-11-16 12:39:05-05:00
Name: Datetime, dtype: datetime64[ns, America/New_York]
model score: 0.959
error 0.8841118357391927
old price =  120.63819885253906
predicted =  120.45653579108429
difference =  -0.1816630614547705


In [46]:
test_stock_w_feature(df2, intra = True)

model score: 0.997
error 0.6804125777533497
old price =  407.602294921875
predicted =  408.02766794391187
difference =  0.4253730220368652


---

With NASDAQ dataframe :

In [47]:
# let's predict he clost price for TSLA after 5 minutes
test_stock(df2, intra = True)

5945   2020-11-16 12:39:07-05:00
Name: Datetime, dtype: datetime64[ns, America/New_York]
model score: 0.997
error 0.6812831213138747
old price =  407.602294921875
predicted =  408.0118030995458
difference =  0.40950817767077297


In [48]:
test_stock_w_feature(df2, intra = True)

model score: 0.997
error 0.6804125777533497
old price =  407.602294921875
predicted =  408.02766794391187
difference =  0.4253730220368652


In [49]:
# Do we same the same gain if use a different modeel: Random Forest ?

In [50]:
def test_stock_grad(dfx, intra = True):
    """
    predict the price for the closing price for the bext inerval. 
    
    inputs:
    ------
    - stock: string, symobol for the stick, eg. 'TSLA'
    - intra : Boolean, if True then the period and interval should be 1m, 5m, etc, if False it should be 1d, 7d, etc
    - period : total period
    - interval : interval for the data
    
    """

    df = dfx.copy()
    df['day_of_the_week'] = df['Datetime'].dt.dayofweek



    
    #print(len(df))
    print(df.tail(1)['Datetime'])
    old = df.tail(1).loc[:,'Open'].to_numpy()[0]
    
    if intra ==True:
            df['Datetime'] = pd.to_datetime(df['Datetime'])
            df['Datetime']=df['Datetime'].map(dt.datetime.toordinal)
            df['SMA_1'] = df['Adj Close'].rolling(window = 26).mean()
            df['SMA_2'] = df['Adj Close'].rolling(window = 50).mean()
            df['status_sma_lower'] = np.where(df['Adj Close'] < df['SMA_1'],'below_movavg','normal')
            df['status_sma_upper'] = np.where(df['Adj Close'] > df['SMA_2'],'above_movavg','normal')
    
    elif intra ==False:
            df['Date'] = pd.to_datetime(df['Date'])
            df['Date']=df['Date'].map(dt.datetime.toordinal)
#             df['SMA_1'] = df['Adj Close'].rolling(window = 50).mean()
#             df['SMA_2'] = df['Adj Close'].rolling(window = 100).mean()
#             df['status_sma_lower'] = np.where(df['Adj Close'] < df['SMA_1'],'below_movavg','normal')
#             df['status_sma_upper'] = np.where(df['Adj Close'] > df['SMA_2'],'above_movavg','normal')

     
    df['High'] = df['High'].shift(1)
    df['Low'] = df['Low'].shift(1)
    df['Volume'] = df['Volume'].shift(1) 
    df['Middle Band'] =df['Adj Close'].rolling(window=26).mean()
    df['Upper Band'] = df['Middle Band'] + 1.96*df['Adj Close'].rolling(window=20).std()
    df['Lower Band'] = df['Middle Band'] - 1.96*df['Adj Close'].rolling(window=20).std()
    df['status_lower'] = np.where(df['Adj Close'] < df['Lower Band'],'below_ballinger','normal')
    df['status_upper'] = np.where(df['Adj Close'] > df['Upper Band'],'above_ballinger','normal')
    df = df.dropna()
    df = df.drop(columns = ['Close'])
    y = df['Adj Close']
    X = df.drop(columns =['Adj Close','Volume' ,'Low','High'])
    X_train, X_test, y_train, y_test = train_test_split(X,y,shuffle= False)
        # first transformer for the numeric features
    numeric_features = list(set(list(X_train.columns)) - set(['status_lower','status_upper','status_sma_lower','status_sma_upper']))
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler()),
       ])
    # now a taransformer for the categorical features
    categorical_features = ['status_lower']
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))])
    # creating a preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)])

    ridge_model = RandomForestRegressor()
    # include the preprocessor and the model in one pipeline.
    # Now we have a full prediction pipeline.
    reg_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                          ('Regressor', ridge_model)])

    # finally we will pass the pipe line to gridsearchcv to find the optimum paramters for the model
    param_grid = {
        'Regressor__n_estimators':[100],
    }
    search = GridSearchCV(reg_pipeline,param_grid,cv = 5)

    # fitting the model
    search.fit(X_train, y_train)

    # printing the first parameter
    #rint(search.best_params_)
    print("model score: %.3f" % search.score(X_test, y_test))
    X_train = X[:-1]
    y_train = y[:-1]
    X_test = X[-1:]
    y_test = y[-1:]
    search.fit(X_train,y_train)
    y_pred = search.predict(X_train)
    print('error',np.sqrt(mean_squared_error(y_train,y_pred)))
    y_test = search.predict(X_test)[0]
    diff = y_test - old
   
    print('old price = ',old)
    print('predicted = ',y_test)
    print('difference = ', diff)
    



In [96]:
def test_stock_grad_feature(dfx, intra = True):
    """
    predict the price for the closing price for the bext inerval. 
    
    inputs:
    ------
    - stock: string, symobol for the stick, eg. 'TSLA'
    - intra : Boolean, if True then the period and interval should be 1m, 5m, etc, if False it should be 1d, 7d, etc
    - period : total period
    - interval : interval for the data
    
    """


    df = dfx.copy()
    df['f01'] = ((df['Open']/df['Close'].shift(1))-1)*100


    #print(len(df))
    print(df.tail(1)['Datetime'])
    old = df.tail(1).loc[:,'Open'].to_numpy()[0]
    
    if intra ==True:
            df['Datetime'] = pd.to_datetime(df['Datetime'])
            df['Datetime']=df['Datetime'].map(dt.datetime.toordinal)
            df['SMA_1'] = df['Adj Close'].rolling(window = 26).mean()
            df['SMA_2'] = df['Adj Close'].rolling(window = 50).mean()
            df['status_sma_lower'] = np.where(df['Adj Close'] < df['SMA_1'],'below_movavg','normal')
            df['status_sma_upper'] = np.where(df['Adj Close'] > df['SMA_2'],'above_movavg','normal')
    
    elif intra ==False:
            df['Date'] = pd.to_datetime(df['Date'])
            df['Date']=df['Date'].map(dt.datetime.toordinal)
#             df['SMA_1'] = df['Adj Close'].rolling(window = 50).mean()
#             df['SMA_2'] = df['Adj Close'].rolling(window = 100).mean()
#             df['status_sma_lower'] = np.where(df['Adj Close'] < df['SMA_1'],'below_movavg','normal')
#             df['status_sma_upper'] = np.where(df['Adj Close'] > df['SMA_2'],'above_movavg','normal')

     
    df['High'] = df['High'].shift(1)
    df['Low'] = df['Low'].shift(1)
    df['Volume'] = df['Volume'].shift(1) 
    df['Middle Band'] =df['Adj Close'].rolling(window=26).mean()
    df['Upper Band'] = df['Middle Band'] + 1.96*df['Adj Close'].rolling(window=20).std()
    df['Lower Band'] = df['Middle Band'] - 1.96*df['Adj Close'].rolling(window=20).std()
    df['status_lower'] = np.where(df['Adj Close'] < df['Lower Band'],'below_ballinger','normal')
    df['status_upper'] = np.where(df['Adj Close'] > df['Upper Band'],'above_ballinger','normal')
    df = df.dropna()
    df = df.drop(columns = ['Close'])
    y = df['Adj Close']
    X = df.drop(columns =['Adj Close','Volume' ,'Low','High'])
    X_train, X_test, y_train, y_test = train_test_split(X,y,shuffle= False)
        # first transformer for the numeric features
    numeric_features = list(set(list(X_train.columns)) - set(['status_lower','status_upper','status_sma_lower','status_sma_upper']))
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler()),
       ])
    # now a taransformer for the categorical features
    categorical_features = ['status_lower']
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))])
    # creating a preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)])

    ridge_model = RandomForestRegressor()
    # include the preprocessor and the model in one pipeline.
    # Now we have a full prediction pipeline.
    reg_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                          ('Regressor', ridge_model)])

    # finally we will pass the pipe line to gridsearchcv to find the optimum paramters for the model
    param_grid = {
        'Regressor__n_estimators':[100],
    }
    search = GridSearchCV(reg_pipeline,param_grid,cv = 5)

    # fitting the model
    search.fit(X_train, y_train)

    # printing the first parameter
    #rint(search.best_params_)
    print("model training score: %.3f" % search.score(X_train, y_train))
    print("model score: %.3f" % search.score(X_test, y_test))
    X_train = X[:-1]
    y_train = y[:-1]
    X_test = X[-1:]
    y_test = y[-1:]
    search.fit(X_train,y_train)
    y_pred = search.predict(X_train)
    error = np.sqrt(mean_squared_error(y_train,y_pred))
    print('error',error)
    y_test = search.predict(X_test)[0]
    diff = y_test - old
        
    if diff > error :
        print('winner')
        print('old price = ',old)
        print('predicted = ',y_test)
        print('difference = ', diff)

    


In [94]:
## Random forest

In [85]:
# let's predict he clost price for TSLA after 5 minutes
test_stock_grad(df1,intra = True)

417   2020-11-16 12:41:03-05:00
Name: Datetime, dtype: datetime64[ns, America/New_York]
model score: 0.881
error 2.2265566962844026
old price =  408.0950012207031
predicted =  408.5079028320313
difference =  0.4129016113281523


In [97]:
test_stock_grad_feature(df1, intra = True)

417   2020-11-16 12:41:03-05:00
Name: Datetime, dtype: datetime64[ns, America/New_York]
model training score: 0.992
model score: 0.889
error 2.296567439161101


> Again we noticed an improved performance when we include this feature

----

In [83]:
# combin these


def test_stocks(tickers = ['TSLA','AAPL']):
        
    for ticker in tickers:
        df = read_data_enhanced(stock=ticker)
        error, diff = test_stock_grad_feature(df, intra = True)
        
        if diff > 0:
            print(ticker,diff)

        
        
        
        
        

In [84]:
test_stocks(tickers = ['TSLA','AAPL'])

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
5951   2020-11-16 12:51:43-05:00
Name: Datetime, dtype: datetime64[ns, America/New_York]
model training score: 1.000
model score: 0.994
error 0.2854275175645735
TSLA 0.36943115234373636
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
5951   2020-11-16 12:52:10-05:00
Name: Datetime, dtype: datetime64[ns, America/New_York]
model training score: 1.000
model score: 0.981
error 0.06064964255811089
AAPL 0.0696896362304642


In [71]:
lista

NameError: name 'lista' is not defined